In [1]:
%load_ext autoreload
%autoreload 2
import os
import time

import pandas as pd

from config import *
from run_openpose import run_openpose
from openpose_helpers import extract_face_hand_presence

# Step 1: Run Openpose on videos

NOTE: It is easier to run this part of code in an IPython session running in a tmux pane, so that you can detach and submit jobs in the background over the course of several days.

In [3]:
#Edit these parameters in config.py

print('VID_ROTATE: ', VID_ROTATE)
print('OPENPOSE_OUTPUT: ', OPENPOSE_OUTPUT)
print('OPENPOSE_CONDENSED_OUTPUT: ', OPENPOSE_CONDENSED_OUTPUT)

VID_ROTATE:  180
OPENPOSE_OUTPUT:  $GROUP_SCRATCH/Home_Headcam_new/outputs/openpose_raw_json
OPENPOSE_CONDENSED_OUTPUT:  $GROUP_SCRATCH/Home_Headcam_new/outputs/openpose_condensed


In [4]:
def queue_size():
    size = !squeue -u $USER
    return len(size) - 1

def wait_for_space(max_jobs):
    time.sleep(5) #Allow for squeue to refresh properly
    if queue_size() >= max_jobs:
        print('Waiting for space in queue...')
    while queue_size() >= max_jobs:
        time.sleep(600)
    
def run_openpose_on_vids(vid_dir, op_output_dir=OPENPOSE_OUTPUT, condensed_output_dir=OPENPOSE_CONDENSED_OUTPUT, flipped_df_path=None):
    fdf = pd.read_csv(flipped_df_path) if flipped_df_path else None
    vid_files = os.listdir(vid_dir)
    for i, vid_name in enumerate(vid_files):
        print(f'{i+1}/{len(vid_files)}: {vid_name}')
        if fdf and fdf[fdf['video'] == vid_name[:-4]]['right_side_up'].values[0] == 1:
            print(f'skipping {vid_name} since it\'s allocentric')
            continue
            # These "right-side-up" videos are mostly allocentric videos,
            # and thus irrelevant to our egocentric analyses
            
        wait_for_space(40)
        vid_path = os.path.join(vid_dir, vid_name)
        run_openpose(vid_path, op_output_dir, 
                     frame_rotate=VID_ROTATE,
                     condensed_output_dir=condensed_output_dir,
                     keypoint_scale=3)

In [ ]:
run_openpose_on_vids(, op_output_dir=OPENPOSE_OUTPUT, condensed_output_dir=OPENPOSE_CONDENSED_OUTPUT):

# Step 2: Extract face/hand presence

In [ ]:
def extract_face_hand_presence_all(condensed_output_dir=OPENPOSE_CONDENSED_OUTPUT):
    condensed_files = os.listdir(condensed_output_dir)
    for i, vid_df_name in enumerate(condensed_files):
        print(f'{i+1}/{len(condensed_files)}: {vid_df_name}')
        vid_df_path = os.path.join(condensed_output_dir, vid_df_name)
        vid_df = pd.read_json(vid_df_path)
        if 'nose_conf' in vid_df and 'wrist_conf' in vid_df: 
            print('already extracted...continuing')
            continue
        vid_df = extract_face_hand_presence(vid_df)
        vid_df.to_json(vid_df_path)